In [ ]:
import synapseclient
import json

import pandas as pd
import great_expectations as gx

from agoradatatools.gx import GreatExpectationsRunner

context = gx.get_context(project_root_dir='../src/agoradatatools/great_expectations')

from expectations.expect_column_values_to_have_list_length import ExpectColumnValuesToHaveListLength
from expectations.expect_column_values_to_have_list_members import ExpectColumnValuesToHaveListMembers

# Create Expectation Suite for Genes Biodomains Data

## Get Example Data File

In [ ]:
syn = synapseclient.Synapse()
syn.login()


In [ ]:
genes_biodomains_data_file = syn.get("syn51062085").path


## Create Validator Object on Data File

In [ ]:
df = pd.read_json(genes_biodomains_data_file)
nested_columns = ['gene_biodomains']
df = GreatExpectationsRunner.convert_nested_columns_to_json(df, nested_columns)
validator = context.sources.pandas_default.read_dataframe(df)
validator.expectation_suite_name = "genes_biodomains"

## Add Expectations to Validator Object For Each Column

In [ ]:
# ensembl_gene_id
validator.expect_column_values_to_be_of_type("ensembl_gene_id", "str")
validator.expect_column_values_to_not_be_null("ensembl_gene_id")
validator.expect_column_value_lengths_to_equal("ensembl_gene_id", 15)
# checks format and allowed chatacters
validator.expect_column_values_to_match_regex("ensembl_gene_id", "^ENSG\d{11}$")
validator.expect_column_values_to_be_unique("ensembl_gene_id")


In [ ]:
# gene_biodomains
validator.expect_column_values_to_be_of_type("gene_biodomains", "str")
validator.expect_column_values_to_not_be_null("gene_biodomains")
#get JSON schema
with open("../src/agoradatatools/great_expectations/gx/json_schemas/genes_biodomains/gene_biodomains_schema.json", "r") as file:
    gene_biodomains_schema = json.load(file)
validator.expect_column_values_to_match_json_schema("gene_biodomains", json_schema=gene_biodomains_schema)

## Save Expectation Suite

In [ ]:
validator.save_expectation_suite(discard_failed_expectations=False)


## Create Checkpoint and View Results

In [ ]:
checkpoint = context.add_or_update_checkpoint(
    name="agora-test-checkpoint",
    validator=validator,
)
checkpoint_result = checkpoint.run()
context.view_validation_result(checkpoint_result)


## Build Data Docs - Click on Expectation Suite to View All Expectations

In [ ]:
context.build_data_docs()
context.open_data_docs()
